In [2]:
ticker = 'bbas3'

In [3]:
import pandas as pd
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [4]:
url = f'https://opcoes.net.br/listaopcoes/completa?au=False&uinhc=0&idLista=ML&idAcao={ticker}&listarVencimentos=true&cotacoes=true'
response = requests.get(url, verify=False).json()

# proximas datas de vencimento
datas = [[i['value'], i['text']] for i in response['data']['vencimentos']]
datas

[['2026-01-16', '16/01'],
 ['2026-01-23', '23/01'],
 ['2026-01-30', '30/01'],
 ['2026-02-06', '06/02'],
 ['2026-02-13', '13/02'],
 ['2026-02-20', '20/02'],
 ['2026-02-27', '27/02'],
 ['2026-03-13', '13/03'],
 ['2026-03-20', '20/03'],
 ['2026-04-17', '17/04'],
 ['2026-05-15', '15/05'],
 ['2026-06-19', '19/06'],
 ['2026-07-17', '17/07'],
 ['2026-08-21', '21/08'],
 ['2026-09-18', '18/09'],
 ['2026-10-16', '16/10'],
 ['2026-11-19', '19/11'],
 ['2026-12-18', '18/12'],
 ['2027-01-15', '15/01/27'],
 ['2027-02-19', '19/02/27'],
 ['2027-03-19', '19/03/27'],
 ['2027-04-16', '16/04/27'],
 ['2027-05-21', '21/05/27'],
 ['2027-06-18', '18/06/27'],
 ['2027-07-16', '16/07/27'],
 ['2027-08-20', '20/08/27'],
 ['2027-09-17', '17/09/27'],
 ['2027-10-15', '15/10/27'],
 ['2027-11-19', '19/11/27'],
 ['2027-12-17', '17/12/27']]

In [5]:
derivativos = []

for vcto in datas:
    url=f'https://opcoes.net.br/listaopcoes/completa?au=False&uinhc=0&idLista=ML&idAcao={ticker}&listarVencimentos=false&cotacoes=true&vencimentos={vcto[0]}'
    response = requests.get(url).json()

    vcto_text = vcto[1]
    derivativos.extend([ticker, i[0].split('_', 1)[0], i[2], i[3], vcto_text, i[5], i[8]] 
            for i in response['data']['cotacoesOpcoes'])

    colunas = ['ATIVO_OBJ','ATIVO', 'TIPO', 'MOD', 'DT_VCTO', 'STRIKE', 'PRECO']
    opcoes = pd.DataFrame(derivativos, columns=colunas)
    
    # transforma o campo vencimento e datetime
    opcoes['DT_VCTO'] = pd.to_datetime(opcoes['DT_VCTO'],  dayfirst=True)
    

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 16/01, at position 0